In [4]:
import solve_sum_min_max
import re

In [5]:
a = 'min(400, 400*a) + min(400, 400*a) + min(0, 100*a)'.split("+")
a

['min(400, 400*a) ', ' min(400, 400*a) ', ' min(0, 100*a)']

In [10]:
'min(400, 400*a) + min(400, 400*a) + min(0, 100*a)'.replace("a", "alpha")

'min(400, 400*alpha) + min(400, 400*alpha) + min(0, 100*alpha)'

In [9]:
m = re.compile("\d+")
m.findall(a[0])

['400', '400']

In [ ]:
for p in 'min(400, 400*a) + min(400, 400*a) + min(0, 100*a)'.split("+"):
    

In [11]:
"min(400, 400*a) +".split("+")

['min(400, 400*a) ', '']